In [15]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [16]:
# method = METHOD_MITHRAL
method = METHOD_PQ
# method = METHOD_EXACT

In [17]:
feedback_bits = 256
ncodebooks=64
ncentroids=8192*2
train_sam_num = 3000 # 训练集样本数

In [18]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = '/data/hdr/transformer_data/joined'
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'



In [19]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  PQ
running kmeans in subspace 1/64... mse / {var(X_subs), var(X)}: 0.0437, 0.000919
running kmeans in subspace 2/64... mse / {var(X_subs), var(X)}: 0.0543, 0.000212
running kmeans in subspace 3/64... mse / {var(X_subs), var(X)}: 0.0301, 0.00128
running kmeans in subspace 4/64... mse / {var(X_subs), var(X)}: 0.027, 0.00302
running kmeans in subspace 5/64... mse / {var(X_subs), var(X)}: 0.0386, 0.00137
running kmeans in subspace 6/64... mse / {var(X_subs), var(X)}: 0.0626, 0.000335
running kmeans in subspace 7/64... mse / {var(X_subs), var(X)}: 0.0386, 0.00262
running kmeans in subspace 8/64... mse / {var(X_subs), var(X)}: 0.0322, 0.00444
running kmeans in subspace 9/64... mse / {var(X_subs), var(X)}: 0.0514, 0.00136
running kmeans in subspace 10/64... mse / {var(X_subs), var(X)}: 0.107, 0.000561
running kmeans in subspace 11/64... mse / {var(X_subs), var(X)}: 0.0536, 0.00328
running kmeans in subspace 12/64... mse / {var(X_subs), var(X)}: 0.0461, 0.00503
running kmeans 

In [20]:
x_test = np.load(dir_train+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [21]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(dir_result+'/'+method+'fc1_fb%i_cb%i_ct%i.npy' % (feedback_bits, ncodebooks, ncentroids), y_out_last)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[-0.03244272 -0.03679594 -0.60117894 ... -0.11950145  0.05724733
  -0.07599946]
 [-0.2888089  -0.17276917 -0.33206522 ... -0.42260846 -0.0334015
  -0.13690414]
 [ 0.27916265 -0.08636951 -0.33914715 ... -0.4877623   0.15922727
   0.05997378]
 ...
 [ 0.23950377  0.12183951 -0.46803847 ... -0.20590112 -0.24585968
  -0.12415665]
 [ 0.20267767  0.01985958 -0.22017059 ... -0.01185597 -0.01782123
   0.00615104]
 [-0.3284678   0.08359703 -0.4397107  ... -0.31496298 -0.11413561
   0.13504234]]
(32000, 256)
